In [1]:
%run StdPackages.ipynb

No clean-up of work-folder


Load wheels and test databases:

In [2]:
fs = [f"{d['data']}\\test_size1000.gdx", f"{d['data']}\\baselinerun.gdx"] # files
ws = gams.GamsWorkspace() 
g2np = gams2numpy.Gams2Numpy(ws.system_directory)
dbs = {'gms1': ws.add_database_from_gdx(fs[0]), 'gms2': ws.add_database_from_gdx(fs[1]),
       'gpy1': Database.GpyDB(db=fs[0],**{'name': 'testdb1'}), 'gpy2': Database.GpyDB(db=fs[1],**{'name': 'testdb1'})}
db = dbs['gpy1']

# GmsWrite.py

```GmsWrite.py``` is used to write simple lines of gams code. The main functions are: ```write_gpy```.

### 1: ```write_gpy(s=None, c = None, alias = {}, lag = {}, l = "", **kwargs)``` 

The method writes standard variations on ```gpy``` symbols ```s``` to gams. Note that beyond the argument 'l', the inputs are equivalent to the ones used in ```DBWheels_rc``` to subset pandas versions of ```gpy``` symbols. The point of this is that the same inputs can be used to slice pandas objects and write gams code that correspond to this. Important differences compared to the ```DBWheels_rc.rctree_pd``` method are:
* ```write_gpy``` only works with ```gpy``` symbols, not pandas objects (as in ```rctree_pd```). The reason is simply that pandas objects to not have sufficient information to write the corresponding gams code.
* ```write_gpy``` includes a level argument "l": The ```rctree_pd``` do not distinguish between different attributes. 
* ```write_gpy``` does not include a ```pm``` argument: In general, the ```rctree_pd``` might throw errors when domains do match and allows one to use partial matching. The gams writing facility is more straightforward in this regard. 
* ```write_gpy``` allows the user to add a condition as a ```string```, in which case this string is simply copied into the conditional statement; the ```rctree_pd``` does not allow for strings (as it matches pandas indices).
* ```write_gpy``` can use other ```gpy``` in the conditional, including variables; however, we do not include methods for automatically writing expressions like ```var.l[x,y]>0``` in the conditional. 
* ```write_gpy``` includes methods for subsetting scalar variables, ```rctree_pd``` does not. 

Note that for the ```write_gpy``` and ```rctree_pd``` to be equivalent, we need to set ```pm=False```, not use ```string``` inputs in the ```write_gpy``` method, and not include conditionals on ```scalars```.

*Automatic write for different types:*

*Variables:*

In [3]:
write_gpy(db['var'])

'var[i,j]'

In [4]:
write_gpy(db['var'], alias = {'i':'ii'}, lag = {'i': -1}, l = ".L")

'var.L[ii-1,j]'

*Sets:*

In [5]:
write_gpy(db['map'])

'map[i,j]'

In [6]:
write_gpy(db['map'], alias = {'j':'jj'}, lag = {'j': "+2"}, l= ".l")

'map[i,jj+2]'

*Using conditionals:*

In [7]:
write_gpy(db['var'], c = db['map'])

'var[i,j]$(map[i,j])'

In [8]:
write_gpy(db['var'], c = db['map'], alias = {'i':'ii'})

'var[ii,j]$(map[i,j])'

*Using nested conditionals:*

In [9]:
write_gpy(db['var'], c = {'s': db['map'], 'c': db['subset']})

'var[i,j]$(map[i,j]$(subset[j]))'

*Note that nested conditionals is kind of a weird thing here that is simpler to construct using and 'and' tuple:*

In [10]:
write_gpy(db['var'], c = ('and', [db['map'],db['subset']]))

'var[i,j]$((map[i,j] and subset[j]))'

*We can also use mixed inputs, e.g. just add some text at the end:*

In [11]:
write_gpy(db['var'], c = ('and', [db['map'], 'then also this text even though gams will not like this']))

'var[i,j]$((map[i,j] and then also this text even though gams will not like this))'